In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from collections import Counter
from numpy import mean
from numpy import std
from pandas import read_csv
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import SelectKBest,chi2,RFE
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OrdinalEncoder

In [2]:
df = pd.read_csv("train.csv")
new_cols = ['C' + str(i) for i in range(1, 14 + 1)] + ['X' + str(i) for i in range(1, 19 + 2)]
new_cols[-1] = 'Y'
df.columns = new_cols

df.head(5)

,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,...,X11,X12,X13,X14,X15,X16,X17,X18,X19,Y
0,V1,V1,V1,V1,V1,V1,V2,V1,V3,V5,...,0.352,0.073,-0.092,1.098,0.034,1.160,0.401,0.037,0.216,1
1,V1,V1,V1,V1,V1,V1,V1,V1,V2,V2,...,-1.933,-0.536,-0.092,1.098,0.034,0.716,0.401,0.724,0.216,-1
2,V1,V1,V1,V1,V1,V1,V1,V1,V2,V2,...,-0.762,-0.536,-0.092,1.098,0.034,0.716,0.401,0.712,0.216,-1
3,V1,V1,V1,V1,V1,V1,V1,V1,V1,V1,...,-0.505,-0.536,-0.092,0.182,0.034,0.716,0.401,0.393,0.216,1
4,V1,V1,V1,V1,V1,V1,V1,V1,V1,V1,...,0.409,-0.536,-0.092,0.731,0.034,0.716,0.401,0.724,0.216,1


In [3]:
features = df.loc[:, df.columns != 'Y']
numerical_features = df.select_dtypes(include=['float64']).columns
categorical_features = df.select_dtypes(include=['object']).columns
target = df['Y']

In [4]:
from sklearn.impute import SimpleImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

iterative_numerical_imputer = IterativeImputer(max_iter=10)
iterative_numerical_imputer.fit(df[numerical_features])
df[numerical_features] = iterative_numerical_imputer.transform(df[numerical_features])

categorical_imputer = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
categorical_imputer.fit(df[categorical_features])
df[categorical_features] = categorical_imputer.transform(df[categorical_features])

encoded_df = pd.get_dummies(df)
encoded_features = encoded_df.loc[:, encoded_df.columns != 'Y']
encoded_df

C:\Users\Gábor\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\impute\_iterative.py:685: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn("[IterativeImputer] Early stopping criterion not"


,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,...,C12_V1,C12_V2,C12_V3,C13_V1,C13_V2,C13_V3,C13_V4,C13_V5,C14_V1,C14_V2
0,0.549000,-0.738000,1.312000,-2.629,-1.104,0.133,3.189,-0.844000,-0.527,0.820000,...,0,1,0,1,0,0,0,0,0,1
1,2.207000,-0.986000,0.823000,0.501,-0.458,1.425,0.120,2.472000,-0.223,-1.823000,...,0,1,0,1,0,0,0,0,1,0
2,2.207000,-0.986000,-1.092000,0.501,-1.363,0.133,-0.099,1.420000,-0.223,-1.284000,...,0,1,0,1,0,0,0,0,0,1
3,0.549000,-0.118000,1.312000,-1.064,0.189,0.133,0.997,0.689000,-0.375,-0.546000,...,1,0,0,1,0,0,0,0,1,0
4,0.549000,-0.118000,0.348000,-0.282,-1.104,1.425,-0.099,0.616000,-0.680,-1.014000,...,1,0,0,1,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
394,-0.383681,0.130000,-0.125459,2.066,0.447,1.425,-0.648,-0.087371,0.172,0.012301,...,1,0,0,1,0,0,0,0,0,1
395,-0.526892,-0.637898,-1.092000,0.579,-0.328,-1.159,0.339,-0.210310,0.842,0.106467,...,1,0,0,1,0,0,0,0,0,1
396,-0.216824,-0.937403,0.348000,0.892,-1.104,1.425,0.339,-0.310218,0.242,0.271979,...,1,0,0,1,0,0,0,0,0,1
397,-0.368202,-0.614000,-0.127000,0.814,-0.328,1.425,0.339,-0.441217,0.242,0.358153,...,1,0,0,1,0,0,0,0,0,1


In [5]:
encoded_df.to_csv("export.csv", index=False)

In [6]:
X_train, X_test, y_train, y_test = train_test_split(encoded_features, target, test_size=0.3)

In [7]:
"""
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
"""

'\nscaler = StandardScaler()\nX_train = scaler.fit_transform(X_train)\nX_test = scaler.transform(X_test)\n'

In [19]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

classifier = RandomForestClassifier(n_estimators=1000)
#classifier = ExtraTreesClassifier(n_estimators=1000)
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)

cm = confusion_matrix(y_test, y_pred)
print(cm)
print(classification_report(y_test, y_pred))

[[71  1]
 [19 29]]
              precision    recall  f1-score   support

          -1       0.79      0.99      0.88        72
           1       0.97      0.60      0.74        48

    accuracy                           0.83       120
   macro avg       0.88      0.80      0.81       120
weighted avg       0.86      0.83      0.82       120



In [9]:
def evaluate_model(X_train, y_train, X_test, y_test, model):
	cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=1)
	#model.fit(X_train, y_train)
	return cross_val_score(model, X_train, y_train, scoring='accuracy', cv=cv, n_jobs=-1)

models, names = list(), list()
# SVM
models.append(SVC(gamma='auto'))
names.append('SVM')
# KNN
models.append(KNeighborsClassifier())
names.append('KNN')
# Bagging
models.append(BaggingClassifier(n_estimators=1000))
names.append('BAG')
# RF
models.append(RandomForestClassifier(n_estimators=1000))
names.append(' RF')
# ET
models.append(ExtraTreesClassifier(n_estimators=1000))
names.append(' ET')

results = list()
for i in range(len(models)):
	scores = evaluate_model(X_train, y_train, X_test, y_test, models[i])
	results.append(scores)
	print('>%s %.3f (%.3f)' % (names[i], scores.mean(), scores.std()))

>SVM 0.714 (0.043)
>KNN 0.741 (0.039)
>BAG 0.768 (0.050)
> RF 0.781 (0.042)
> ET 0.810 (0.031)


In [10]:
"""
clf = RandomForestClassifier()
clf.fit(encoded_features, target)

plt.figure(figsize=(20,8))
plt.bar(encoded_features.columns, clf.feature_importances_)
plt.xticks(rotation=45)
"""

'\nclf = RandomForestClassifier()\nclf.fit(encoded_features, target)\n\nplt.figure(figsize=(20,8))\nplt.bar(encoded_features.columns, clf.feature_importances_)\nplt.xticks(rotation=45)\n'

In [11]:
#GridSearchCV?
# TODO https://datascience.stackexchange.com/questions/75345/need-help-understanding-data-leakage